In [1]:
import sys
sys.path.append('../../assets/python/')
import itertools as it
import math
import numpy as np
import plotly.graph_objects as go
import tfb

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

COLOR_LIST = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

In [46]:
# functions for calculating initiative probabilities
def init_prob_a_above_b_given_ia(i_a, m_b):
    """Probability of character b rolling below character a's initiative roll.

    i_a: character a's initiative value.
    m_b: character b's initiative bonus.
    """
    return min(20, max(0, (i_a - m_b - 1)))/20

def init_prob_a_below_b_given_ia(i_a, m_b):
    """Probability of character b rolling above character a's initiative roll.

    i_a: character a's initiative value.
    m_b: character b's initiative bonus.
    """
    return min(20, max(0, 20 - (i_a - m_b)))/20

def init_prob_a_ties_b_given_ia(i_a, m_b):
    """Probability of character b tying character a's initiative roll.

    i_a: character a's initiative value.
    m_b: character b's initiative bonus.
    """
    if 20 + m_b < i_a:
        return 0
    elif 1 + m_b > i_a:
        return 0
    else:
        return 1/20

def init_prob_a_above_b(m_a, m_b):
    """Probability of character b rolling below character a's initiative roll.

    i_a: character a's initiative value.
    m_b: character b's initiative bonus.
    """
    dm = m_a - m_b
    if dm >  20: return 1.0
    if dm < -20: return 0.0
    if dm >   0: return 1 - (20 - dm)*(21 - dm)/800
    return (19 + dm)*(20 + dm)/800

def init_prob_a_below_b(m_a, m_b):
    """Probability of character b rolling above character a's initiative roll.

    i_a: character a's initiative value.
    m_b: character b's initiative bonus.
    """
    return init_prob_a_above_b(m_b, m_a)

def init_prob_a_ties_b(m_a, m_b):
    """Probability of character b tying character a's initiative roll.

    i_a: character a's initiative value.
    m_b: character b's initiative bonus.
    """
    dm = m_a - m_b
    if dm >  20: return 0
    if dm < -20: return 0
    if dm >=  0: return (20 - dm)/400
    return (20 + dm)/400

def init_prob_position(combatants, combatant=None, position=1):
    if not combatant:
        k_1 = list(combatants.keys())[0]
    else:
        k_1 = combatant
    
    def eval_state(pos, states, nc=None, na=0, nb=0, nt=0):
        if not nc: nc = len(states)
        if na >= pos: return 0
        if nb > nc - pos: return 0
        if not states: return 1.0/nt
        
        c_states = states.pop()
        prob = 0
        for s, p in c_states:
            match s:
                case 'a':
                    prob += p*eval_state(pos, states, nc, na+1, nb, nt)
                case 'b':
                    prob += p*eval_state(pos, states, nc, na, nb+1, nt)
                case 't':
                    prob += p*eval_state(pos, states, nc, na, nb, nt+1)
        
        states.append(c_states)
        return prob

    # calculate the probability for each initiative roll
    p_initiative = []
    for i in range(1+combatants[k_1], 21+combatants[k_1]):
        # construct states and probabilities for each character
        states = []
        for c in combatants:
            c_states = []
            if c == k_1:
                pa = 0.00
                pb = 0.00
                pt = 0.05
            else:
                pa = init_prob_a_below_b_given_ia(i, combatants[c])
                pb = init_prob_a_above_b_given_ia(i, combatants[c])
                pt = init_prob_a_ties_b_given_ia(i, combatants[c])
            if pa > 0: c_states.append(('a', pa))
            if pb > 0: c_states.append(('b', pb))
            if pt > 0: c_states.append(('t', pt))
            states.append(c_states)

        # construct all possible state combinations and calculate probabilities
        p_initiative.append(eval_state(position, states))

    return sum(p_initiative)

def eval_weights(rolls):
    """Calculates the weight from the given initiative rolls
    """
    w = 1
    tw = 1
    for i in range(len(rolls)-1):
        if rolls[i] < rolls[i+1]:
            return 0
        elif rolls[i] > rolls[i+1]:
            w *= math.factorial(tw)
            tw = 1
        else:
            #return 0
            tw += 1
    w *= math.factorial(tw)
    return 1/w

def init_prob_order( modifiers, depth=0, max_depth=None, rolls=[] ):
    """Calculates the probability for an explicit order given the character initiative modifiers
    """
    if depth == 0:
        rolls = [0]*len(modifiers)
        max_depth = len(modifiers)-1
    
    m = modifiers[depth]

    val = 0
    i0 = 1+max(modifiers[depth:]) # avoid checking combos that can't work
    i1 = min(rolls[depth-1], 20+m) if depth > 0 else 20+m
    
    for i in range(i0, i1+1):
        rolls[depth] = i
        if depth < max_depth:
            dv = init_prob_order(modifiers, depth+1, max_depth, rolls)
        else:
            dv = eval_weights(rolls)
        val += dv
    if depth == 0:
        val = val / 20**len(modifiers)
    return val

In [36]:
# alternate (slower) functions
def init_prob_position(combatants, combatant=None, position=1):
    n_combatants = len(combatants)

    if not combatant:
        k_1 = list(combatants.keys())[0]
        k_other = list(combatants.keys())[1:]
    else:
        k_1 = combatant
        k_other = [k for k in combatants.keys() if k != combatant]

    # generate possible combinations resulting in the desired position
    combos = []
    for n_ahead in range(0, position):
        for n_behind in range(0, n_combatants - position + 1):
            #n_ties = n_others - n_ahead - n_behind
            for k_ahead in it.combinations(k_other, n_ahead):
                k_remaining = [k for k in k_other if k not in k_ahead]
                for k_behind in it.combinations(k_remaining, n_behind):
                    k_tied = [k_1] + [k for k in k_remaining if k not in k_behind] 
                    combos.append((list(k_ahead), list(k_tied), list(k_behind)))

    
    # calculate the probability for each initiative roll
    p_initiative = []
    for i in range(1+combatants[k_1], 21+combatants[k_1]):
        probs_a = {k: init_prob_a_below_b_given_ia(i, combatants[k]) for k in combatants}
        probs_b = {k: init_prob_a_above_b_given_ia(i, combatants[k]) for k in combatants}
        probs_t = {k: init_prob_a_ties_b_given_ia(i, combatants[k]) for k in combatants}
        
        # calculate the probability across all combinations
        p_total = 0
        for k_ahead, k_tied, k_behind in combos:
            probs = [probs_a[k] for k in k_ahead] + [probs_b[k] for k in k_behind] + [probs_t[k] for k in k_tied]
            if 0 in probs: continue

            prob = math.prod(probs)/len(k_tied)
            p_total += prob
        
        p_initiative.append(p_total)


    return sum(p_initiative)

In [3]:
# Fig 1: Shows the probability of character A's initiative roll being higher than B's, lower than B's, or tying B's
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='$i_A - m_B$',
            range=[-1, 22],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[-0.01,1.01],
            tickformat='.1f',
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=0.60,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

i_a = list(range(-1, 23))
m_b = 0
y = [init_prob_a_above_b_given_ia(x, m_b) for x in i_a]
fig.add_trace(go.Scatter(
    x=i_a,
    y=y,
    mode='lines+markers',
    name=f'A > B',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>A > B</b><br>'+
        'i_a - m_b %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

i_a = list(range(-1, 23))
m_b = 0
y = [init_prob_a_below_b_given_ia(x, m_b) for x in i_a]
fig.add_trace(go.Scatter(
    x=i_a,
    y=y,
    mode='lines+markers',
    name=f'A < B',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>A < B</b><br>'+
        'i_a - m_b %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

i_a = list(range(-1, 23))
m_b = 0
y = [init_prob_a_ties_b_given_ia(x, m_b) for x in i_a]
fig.add_trace(go.Scatter(
    x=i_a,
    y=y,
    mode='lines+markers',
    name=f'A = B',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>A = B</b><br>'+
        'i_a - m_b %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-relative-probs-given-ia-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-relative-probs-given-ia-small')

In [13]:
# Fig 2: Shows the probability of character A rolling higher than B, lower than B, or tying B
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='$m_A - m_B$',
            range=[-21, 21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[-0.01,1.01],
            tickformat='.1f',
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=0.60,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

m_a = list(range(-25, 26))
m_b = 0
y = [init_prob_a_above_b(x, m_b) for x in m_a]
fig.add_trace(go.Scatter(
    x=m_a,
    y=y,
    mode='lines',
    name=f'A > B',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>A > B</b><br>'+
        'i_a - m_b %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

y = [init_prob_a_below_b(x, m_b) for x in m_a]
fig.add_trace(go.Scatter(
    x=m_a,
    y=y,
    mode='lines',
    name=f'A < B',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>A < B</b><br>'+
        'i_a - m_b %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

y = [init_prob_a_ties_b(x, m_b) for x in m_a]
fig.add_trace(go.Scatter(
    x=m_a,
    y=y,
    name=f'A = B',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>A = B</b><br>'+
        'i_a - m_b %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-relative-probs-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-relative-probs-small')

In [15]:
# Fig 3: Shows the probability of characters A and B going first
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='$m_A - m_B$',
            range=[-22, 22],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[-0.01,1.01],
            tickformat='.1f',
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=0.60,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

m_a = list(range(-20, 21))
m_b = 0
y = [init_prob_a_above_b(x, m_b) + 0.5*init_prob_a_ties_b(x, m_b) for x in m_a]
fig.add_trace(go.Scatter(
    x=m_a,
    y=y,
    mode='lines',
    name=f'A first',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>A first</b><br>'+
        'i_a - m_b %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

y = [init_prob_a_below_b(x, m_b) + 0.5*init_prob_a_ties_b(x, m_b) for x in m_a]
fig.add_trace(go.Scatter(
    x=m_a,
    y=y,
    mode='lines',
    name=f'B first',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        f'<b>B first</b><br>'+
        'i_a - m_b %{x:.0f}<br>'+
        'probability %{y:.1%}<extra></extra>',
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-relative-order-probs-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-relative-order-probs-small')

In [47]:
# Fig 4: Shows the probability of each initiative order for an encounter with three characters
import itertools as it

combatants = {
    'A': 2,
    'B': 5,
    'C': 0,
}
order = ''.join(combatants.keys())

results = {}
for io in it.permutations(order, len(order)):
    M_list = [combatants[k] for k in io]
    results[''.join(io)] = init_prob_order(M_list)


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='Order',
            #range=[-22, 22],
            #tick0=0, dtick=5,
            #minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[-0.01,0.41],
            tickformat='.1f',
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.05),
        ),
    )
)

fig.add_trace(go.Bar(
    x=list(results.keys()),
    y=list(results.values()),
    #mode='lines',
    #name=f'B first',
    #line_color=COLOR_LIST[i],
    hovertemplate=
        'order %{x}<br>'+
        'probability %{y:.1%}<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-explicit-order-probs-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-explicit-order-probs-small')

In [21]:
# Fig 5: Shows the probability of each initiative position for a PC relative to four monsters
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='initiative order',
            range=[0, 6],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[0,1.02],
            #tickformat='.0%'
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

n_enemies = 4
enemy_init_mod = 0
init_mods = [-8, -4, 0, 4, 8]
for init_mod in init_mods:
    combatants = {
        'A': init_mod, 
        'B': 0, 'C': 0, 'D': 0, 'E': 0,
    }
    order = np.array(list(range(1, len(combatants)+1)))
    probs = np.array([init_prob_position(combatants, combatant='A', position=i) for i in order])
    p_ave = np.dot(order, probs)
    p_sig = np.sqrt(np.dot((order - p_ave)**2, probs))
    fig.add_trace(go.Scatter(
        x=order,
        y=probs,
        name=f'init bonus = {init_mod:.0f}; mean={p_ave:.1f}, std={p_sig:.1f}',
        hovertemplate=
            f'<b>init bonus = {init_mod:.0f}</b><br>'+
            'position %{x:.0f}<br>'+
            'probability %{y:.1%}<extra></extra>',
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-general-position-example-use-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-general-position-example-use-small')

In [38]:
# Fig 5 (alt): Shows the average position and standard deviation as a function of initiative bonus
combatants = {
    'A': 0, 
    'B': 0, 'C': 0, 'D': 0, 'E': 0, 
    #'F': 0, 'G': 0, 'H': 0, 'I': 0,
}

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='initiative bonus',
            range=[-21, 21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='position',
            range=[0,len(combatants)+1],
            #tickformat='.0%'
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=1.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

init_mods = list(range(-20,21))
p_ave_list = []
p_sig_list = []
for init_mod in init_mods:
    combatants['A'] = init_mod
    order = np.array(list(range(1, len(combatants)+1)))
    probs = np.array([init_prob_position(combatants, combatant='A', position=i) for i in order])
    p_ave = np.dot(order, probs)
    p_var = np.dot((order - p_ave)**2, probs)
    p_ave_list.append(p_ave)
    p_sig_list.append(math.sqrt(p_var))


fig.add_trace(go.Scatter(
    x=init_mods,
    y=p_ave_list,
    name='position - mean',
    hovertemplate=
        'initiative bonus %{x:.0f}<br>'+
        'average position %{y:.2f}<extra></extra>',
))

fig.add_trace(go.Scatter(
    x=init_mods,
    y=p_sig_list,
    name='position - sigma',
    hovertemplate=
        'initiative bonus %{x:.0f}<br>'+
        'position sigma %{y:.2f}<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-general-position-example-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-general-position-example-small')

In [20]:
# Fig 5 (alt): Shows the probabilities of three characters ending up in each of the three possible initiative positions
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='position',
            range=[0.5, 3.5],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[0,0.6],
            #tickformat='.0%'
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

combatants = {'A': 2, 'B': 4, 'C': 0}

for combatant in combatants.keys():
    order = list(range(1, len(combatants)+1))
    probs = [init_prob_position(combatants, combatant=combatant, position=i) for i in order]
    fig.add_trace(go.Scatter(
        x=order,
        y=probs,
        name=f'{combatant}; ave={np.dot(order, probs):.1f}',
        hovertemplate=
            f'<b>{combatant}</b><br>'+
            'initiative order %{x:.0f}<br>'+
            'probability %{y:.1%}<extra></extra>',
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-general-position-probs-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-general-position-probs-small')

In [27]:
# Fig 6: Shows the scaling of the Explicit Order and General Position calculations
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='combatants',
            range=[0, 21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='calculations',
            type='log',
            #range=[0,1.02],
            #tickformat='.0%'
            #tick0=0, dtick=0.2,
            #minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

x = list(range(2, 9))
fig.add_trace(go.Scatter(
    x=x,
    y=(20**np.array(x)),
    name='explicit order',
    hovertemplate=
        '<b>explicit order</b><br>'+
        'combatants %{x:.0f}<br>'+
        'combos %{y:.0f}<extra></extra>',
))


x = list(range(2, 21))
y = []
for n_combatants in x:
    position = int(n_combatants/2)
    n_combos = 0
    for n_a in range(0, position):
        for n_b in range(0, n_combatants - position + 1):
            n_combos += math.comb(n_combatants-1, n_a)*math.comb(n_combatants-n_a-1, n_b)

    n_combos *= 20
    y.append(n_combos)

fig.add_trace(go.Scatter(
    x=x,
    y=y,
    name='general position',
    hovertemplate=
        '<b>general position</b><br>'+
        'combatants %{x:.0f}<br>'+
        'combos %{y:.0f}<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-calculation-scaling-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-calculation-scaling-small')